# Data Science Fundamentals
## Group Project - Predicting the Outcome of a Car Accident in Zurich City
#### Group Members:
- Elia Locher (21-941-000)
- Gabriel Oget (22-610-893)
- Marco Molnar (21-917-315)
#### Content:
1. Feature Engineering and Data Preprocessing
2. Data Exploration and Analysis
3. Model Training
4. Results and Concluion
5. Sources

### Required Libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely as shp
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import math
from datetime import datetime, timedelta, date
import seaborn as sns
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from dateutil.easter import easter
import sklearn as sk
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, make_scorer, precision_recall_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras_tuner import RandomSearch, Objective

## 1. Feature Engineering and Data Preprocessing

### 1.1 Importing the Main Dataset

In [ ]:
# load data and only select the columns we need
accidents_raw = pd.read_csv('roadtrafficaccidentlocations.csv', usecols=['AccidentUID', 'AccidentType', 'AccidentSeverityCategory', 'AccidentInvolvingPedestrian', 'AccidentInvolvingBicycle', 'AccidentInvolvingMotorcycle', 'RoadType', 'AccidentYear', 'AccidentMonth', 'AccidentWeekDay', 'AccidentHour', 'AccidentLocation_CHLV95_E', 'AccidentLocation_CHLV95_N'])
# exclude 2011, because we don't have traffic volume data for that year
accidents_raw = accidents_raw.loc[accidents_raw['AccidentYear'] >= 2012].reset_index(drop=True)

In [ ]:
accidents_raw.head(3)

### 1.2 Data Preprocessing: Date and Time

For further Feature Engineering it is useful if we have a Date-Time column. Given the year, month wnd weekday and assuming a chronological order of the observations, we can calculate the exact Date. The following function also assumes there was at least one accident per week, which seems plausable as there are 55000 accidents and only a total of about 4000 days.

In [ ]:
# Format the Weekday column
accidents_raw['AccidentWeekDay'] = accidents_raw['AccidentWeekDay'].str[-1].astype(int)

# Setting the start date
year = accidents_raw['AccidentYear'][0] # 2011
month = accidents_raw['AccidentMonth'][0] # 1
# weekday is the information we have (1 to 7 for Monday to Sunday)
weekday = accidents_raw['AccidentWeekDay'][0] # 6
# monthday is the information we want to calculate (1 to 31 for the day of the month)
monthday = accidents_raw['AccidentMonth'][0] # 1

# Initialize the DateTime column
accidents_raw['DateTime'] = datetime(1900, 1, 1, 0, 0, 0)

# Iterate over all rows and calculate the DateTime
for i in tqdm(accidents_raw.index):
    # The hour will just stay the same
    hour = accidents_raw['AccidentHour'][i]
    
    # Is the weekday the same as before?
    if accidents_raw['AccidentWeekDay'][i] != weekday:
        new_weekday = accidents_raw['AccidentWeekDay'][i]
        # calculate the distance between the two weekdays and add it to the monthday
        distance = new_weekday - weekday
        if distance < 0:
            distance += 7
        monthday += distance
        weekday = new_weekday
        
        # if new month has started, reset monthday and reload year and month, becase we only have to recheck the year if the month has changed
        if accidents_raw['AccidentMonth'][i] != month:
            monthday = 1
            year = accidents_raw['AccidentYear'][i]
            month = accidents_raw['AccidentMonth'][i]
            
    accidents_raw.loc[i, 'DateTime'] = datetime(year, month, monthday, hour)

In [ ]:
accidents_raw.head(3)

### 1.3 Data Preprocessing: Coordinates

It is also useful to have a column with preformatted coordinate points for each observation, replacing the 'E' and 'N' Coordinates. By the way: All our datasets that use geospacial information, use a specific swiss coordinate reference system, so we dont need to worry about different systems.

In [ ]:
# using a list comprehension to create the geometry column with the shapely Point objects
accidents_raw['geometry'] = [shp.Point(x, y) for x, y in zip(accidents_raw['AccidentLocation_CHLV95_E'], accidents_raw['AccidentLocation_CHLV95_N'])]
# also create a GeoDataFrame and drop the old columns
accidents_raw = gpd.GeoDataFrame(accidents_raw, geometry='geometry')#.drop(['AccidentLocation_CHLV95_E', 'AccidentLocation_CHLV95_N'], axis=1)

In [ ]:
accidents_raw.head(3)

In [ ]:
accidents_raw['geometry'].plot(figsize=(30, 30), alpha=0.2, markersize=4)

### 1.4 Feature Engineering: Traffic Volume

The first real feature we will implement here is the traffic volume, measured by automated counting stations all over Zurich City.

In [ ]:
column_data_types = {'HNr': str, 'D2ID': str}

def read_traffic_data(year):
    filename = f"Datasets/Verkehrsaufkommen/sid_dav_verkehrszaehlung_miv_OD2031_{year}.csv"
    return pd.read_csv(filename, dtype=column_data_types, usecols=['MSID', 'MSName', 'ZSID', 'ZSName', 'EKoord', 'NKoord', 'MessungDatZeit', 'AnzFahrzeuge', 'Richtung'])

trafficdic = {}

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(read_traffic_data, i): i for i in range(2012, 2023)}
    for future in tqdm(as_completed(futures), total=len(futures)):
        year = futures[future]
        trafficdic[year] = future.result()
        
traffic_coords = trafficdic[2022][['EKoord', 'NKoord']].drop_duplicates().reset_index(drop=True)

In [ ]:
# First look at the first traffic frequency dataframe
trafficdic[2012].info()

We see that our interesting variable 'AnzFahrzeuge' which is the measurement of the traffic frequency has a lot of missing values. Just look at the observation with missing values for the variable 'AnzFahrzeuge' below:

In [ ]:
trafficdic[2012][trafficdic[2012]["AnzFahrzeuge"].isna()]

For this reason we have to impute this variable. We will show two different methods for imputing the variable 'AnzFahrzeuge'.
But before we are going to impoute the variable. We want to discuss how we want to build this dataframe up. The first point is that we are going to calculate the distance between each accident and the different counting stations. To compute the distance between two points p and q we will use the so called 'Euclidean distance':
$$ d(p, q)^2 = (q_1 - p_1)^2 + (q_2 - p_2)^2 $$

As a next step we are going to evaluate the six nearest counting stations, so we will calculate the six smallest distances. The notation is the following:
+ 'CountingStation_MinDistance' is the station which measures traffic frequency, which is AnzFahrzeuge' is the traffic measurement from the nearest counting station to the accident
+ 'AnzFahrzeuge_2thMinDistance' is the traffic measurement from the second nearest counting station to the accident and so on until the variable 'AnzFahrzeuge_6thMinDistance'

#### 1.4.2 Traffic: Data Imputation

In [ ]:
def get_nth_smallest_info(group, n):
    if len(group) >= n:
        nth_smallest_row = group.nsmallest(n, 'Distanz').iloc[-1]
        return pd.Series({
            f'{n}thMinDistance': nth_smallest_row['Distanz'],
            f'CountingStation_{n}thMinDistance': nth_smallest_row['CountingStation'],
            f'AnzFahrzeuge_{n}thMinDistance': nth_smallest_row['AnzFahrzeuge'],
            f'AnzFahrzeuge_{n}thMinDistance_-1h': nth_smallest_row['AnzFahrzeuge_-1h'],
            f'AnzFahrzeuge_{n}thMinDistance_-2h': nth_smallest_row['AnzFahrzeuge_-2h']
        })
    return pd.Series({
        f'{n}thMinDistance': None,
        f'CountingStation_{n}thMinDistance': None,
        f'AnzFahrzeuge_{n}thMinDistance': None,
        f'AnzFahrzeuge_{n}thMinDistance_-1h': None,
        f'AnzFahrzeuge_{n}thMinDistance_-2h': None
    })
    
def prepare_traffic_data(df_traffic,year):
    zurich = accidents_raw.copy()
    zurich["Date"] = accidents_raw["DateTime"]
    df_accident = zurich[zurich['AccidentYear'] == year][["Date", "AccidentUID", "AccidentLocation_CHLV95_E", "AccidentLocation_CHLV95_N"]]

    df_traffic["Date"] = pd.to_datetime(df_traffic["MessungDatZeit"])
    df_traffic['Year'] = df_traffic['Date'].dt.year
    df_traffic['Weekday'] = df_traffic['Date'].dt.weekday  # Monday is 0, Sunday is 6
    df_traffic['Daytime'] = df_traffic['Date'].dt.time
    df_traffic["CountingStation"] = df_traffic["ZSName"] + " " + df_traffic["Richtung"]
    df_traffic['AnzFahrzeuge_-1h'] = df_traffic.groupby('CountingStation')['AnzFahrzeuge'].shift(1)
    df_traffic['AnzFahrzeuge_-2h'] = df_traffic.groupby('CountingStation')['AnzFahrzeuge'].shift(2)
    
    missings = df_traffic[df_traffic["AnzFahrzeuge"].isna()].value_counts('CountingStation').reset_index()
 
    # Schwellenwert berechnen (50% von 8760) --> 8760 = 365*24; Maximal number of observations per station and year
    threshold = 0.50 * 8760

    # Filtern des DataFrames
    filtered_stations = missings[missings['count'] > threshold]

    # Anzeigen der gefilterten 'CountingStation'
    filtered_stations

    stations_list = filtered_stations['CountingStation'].tolist()
    df_traffic = df_traffic.loc[~df_traffic['CountingStation'].isin(stations_list)]
    
    # Gruppieren der Werte, sodass jede Gruppe eine Messung an einer CountingStation zu einer spezifischen Tageszeit an einem Wochentag darstellt  
    # Die fehlenden Werte werden durch den (jahres) Gruppendurchschnitt imputiert
    imputation = df_traffic.groupby(['CountingStation', 'Weekday', 'Daytime'])['AnzFahrzeuge'].mean().round(0).reset_index()
    imputation.rename(columns={"AnzFahrzeuge":"ImputedAnzFahrzeuge"},inplace=True)
    
    df_traffic = pd.merge(df_traffic,imputation,on=['CountingStation','Weekday','Daytime'],how="left")
    df_traffic["AnzFahrzeuge"] = df_traffic["AnzFahrzeuge"].fillna(df_traffic["ImputedAnzFahrzeuge"])
    df_traffic = df_traffic.sort_values(by=["CountingStation","Date"])
    df_traffic['AnzFahrzeuge_-1h'] = df_traffic.groupby('CountingStation')['AnzFahrzeuge'].shift(1)
    df_traffic['AnzFahrzeuge_-2h'] = df_traffic.groupby('CountingStation')['AnzFahrzeuge'].shift(2)
    df_traffic.drop(columns=["MSID","MSName","ZSID","ZSName","Richtung","MessungDatZeit","Weekday","Daytime"], inplace=True)
    df_traffic['Year'] = df_traffic['Date'].dt.year
    df_traffic = df_traffic[["Date","Year","EKoord","NKoord","CountingStation","AnzFahrzeuge","AnzFahrzeuge_-1h","AnzFahrzeuge_-2h"]]
    
    merged = pd.merge(df_traffic,df_accident,on="Date",how="left")
    merged['Distanz'] = np.sqrt((merged['AccidentLocation_CHLV95_E'] - merged['EKoord'])**2 +
                               (merged['AccidentLocation_CHLV95_N'] - merged['NKoord'])**2)
    
    min_info_per_accident = merged.groupby('AccidentUID').apply(lambda x: x.nsmallest(1, 'Distanz')).reset_index(drop=True)

    
    # Rename columns for the minimum distance and other related fields
    min_info_per_accident.rename(columns={
        'Distanz': 'MinDistance', 
        'CountingStation': 'CountingStation_MinDistance'}, inplace=True)

    # Merge the min info back into the original dataframe
    merged = merged.merge(min_info_per_accident[['AccidentUID', 'MinDistance', 'CountingStation_MinDistance']], on='AccidentUID')
    merged = merged.sort_values(by=["CountingStation","Date"])
    # Function to get the nth smallest distance and corresponding information for each AccidentUID
    
    # Applying the function for 2nd to 6th smallest distances
    for i in range(2, 4):
        nth_min_info = merged.groupby('AccidentUID').apply(lambda group: get_nth_smallest_info(group, i)).reset_index()
        merged = merged.merge(nth_min_info, on='AccidentUID')

    merged = merged[
        (merged['Distanz'] == merged['MinDistance']) &
        (merged['CountingStation'] == merged['CountingStation_MinDistance'])]
    
    columns_to_fill_withzero = ['AnzFahrzeuge_-1h','AnzFahrzeuge_2thMinDistance_-1h','AnzFahrzeuge_3thMinDistance_-1h',
                                'AnzFahrzeuge_-2h','AnzFahrzeuge_2thMinDistance_-2h','AnzFahrzeuge_3thMinDistance_-2h']

    # Durch die Liste iterieren und NaN-Werte mit 0 auffüllen
    for col in columns_to_fill_withzero:
        merged[col] = merged[col].fillna(0)

    # to account for data leackage, we dont select the traffic volume as counted in the same hour the accident occurred in
    df_imputed = merged[["Date","AccidentUID","CountingStation_MinDistance","MinDistance","AnzFahrzeuge_-1h","AnzFahrzeuge_-2h",
                                  "CountingStation_2thMinDistance","2thMinDistance","AnzFahrzeuge_2thMinDistance_-1h","AnzFahrzeuge_2thMinDistance_-2h",
                                  "CountingStation_3thMinDistance","3thMinDistance","AnzFahrzeuge_3thMinDistance_-1h","AnzFahrzeuge_3thMinDistance_-2h"
    ]]
    
    return df_imputed


In [ ]:
for y in tqdm(range(2012, 2023)):
    trafficdic[y] = prepare_traffic_data(trafficdic[y],y)

In [ ]:
traffic = pd.concat(trafficdic.values(), ignore_index=True)
accidents_raw = pd.merge(accidents_raw, traffic, on=['AccidentUID'], how='left')

In [ ]:
accidents_raw.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')

### 1.5 Feature Engineering: Weather

We will also use the weather data collected by the three weather stations in Zurich to create lagged features for temperature, rainfall, etc.

In [ ]:
# Import all Datasets and storing them in a dictionary
weatherdic = {}

def read_weather_data(year):
    filename = f"Datasets/Wetter/ugz_ogd_meteo_h1_{year}.csv"
    return pd.read_csv(filename, usecols=['Datum', 'Standort', 'Parameter', 'Wert'])

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(read_weather_data, i): i for i in range(2011, 2023)}
    for future in tqdm(as_completed(futures), total=len(futures)):
        year = futures[future]
        weatherdic[year] = future.result()

In [ ]:
weatherdic[2011].head(3)

All values of 'Status' are 'Bereinigt' for all years, meaning there is no skewed data due to for example technical failures.

#### 1.5.1 Weather: Transforming Long to Wide Dataframe

In [ ]:
# Function that tranforms long DF into wide DF and format datetime
def transformation (df):
    # Transform Datum into str...
    #df.Datum.astype('str')
    # and then into datetime
    df['DateTime'] = pd.to_datetime(df['Datum'])
    df.drop(columns=["Datum"], inplace=True)
    
    # Transformation from long to wide format
    df = df.pivot(
        # We drop Intervall, Einheit and Status, as they are consistent over all dataframes and parameters
        index=["DateTime", "Standort"], columns=["Parameter"], values="Wert").reset_index() #index ist der Identifier

    return df #return the wide DF

In [ ]:
# apply the function to all Dataframes
for i in range(2011, 2023):
    weatherdic[i] = transformation(weatherdic[i])

# Concat all the wide Dataframes
weather_wide = pd.concat([df for df in weatherdic.values()], ignore_index=True)

# Rename the column Parameter by adding information about the Einheit (unit) of each Parameter
weather_wide.rename(columns={'T': 'Temp (°C)', 'Hr': 'Hr (%Hr)', 'p': 'p (hPa)', 'RainDur': 'RainDur (min)', 'StrGlo': 'StrGlo (W/m2)',
                             'WD': 'WD (°)', 'WVv': 'WVv (m/s)', 'WVs': 'WVs (m/s)'}, inplace=True)

In [ ]:
weather_wide.head(3)

#### 1.5.2 Weather: Getting the Mean for each Parameter

In [ ]:
# First look how many Weatherstations we have
weather_wide['Standort'].value_counts()

In [ ]:
# create three separate DF's for each weatherstation
rosengarten = weather_wide.query("Standort == 'Zch_Rosengartenstrasse'")
schimmel = weather_wide.query("Standort == 'Zch_Schimmelstrasse'")
stampfenbach = weather_wide.query("Standort == 'Zch_Stampfenbachstrasse'")

In [ ]:
# merge the three DF's by date in order to have the hourly weather variable of each station at a given date in the same row
weather_wide = rosengarten.merge(schimmel,on='DateTime', how ='outer').merge(stampfenbach,on='DateTime',how ='outer')#.drop(columns=['Standort_x','Standort_y','Standort'])

In [ ]:
# sort the values by Date
weather_wide.sort_values(by='DateTime', inplace=True)
weather_wide = weather_wide.reset_index()

In [ ]:
# calculate for each weather variable the row wise mean -> we get the mean of each weather variable from the three stations
# by default the .mean()-method ignores missing values (e.g on 01.01.2012 0:00 we only have the weather variable of the weather stations
# at Schimmelstrasse and Stampfenbachstrasse but at Rosengartenstrasse we have a NaN. The .mean()-methode therefore takes the value of
# the Schimmelstrasse and Stampfenbachstrasse and divides it by two. As a result we get the correct mean of a weather variable at a given hour.)
weather_wide['Hr (%Hr)_mean'] = weather_wide[['Hr (%Hr)_x', 'Hr (%Hr)_y', 'Hr (%Hr)']].mean(axis=1)
weather_wide['RainDur (min)_mean'] = weather_wide[['RainDur (min)_x', 'RainDur (min)_y', 'RainDur (min)']].mean(axis=1)
weather_wide['StrGlo (W/m2)_mean'] = weather_wide[['StrGlo (W/m2)_x', 'StrGlo (W/m2)_y', 'StrGlo (W/m2)']].mean(axis=1)
weather_wide['Temp (°C)_mean'] = weather_wide[['Temp (°C)_x', 'Temp (°C)_y', 'Temp (°C)']].mean(axis=1).drop(columns=['Temp (°C)_x', 'Temp (°C)_y', 'Temp (°C)'])
weather_wide['WD (°)_mean'] = weather_wide[['WD (°)_x', 'WD (°)_y', 'WD (°)']].mean(axis=1).drop(columns=['WD (°)_x', 'WD (°)_y', 'WD (°)'])
weather_wide['WVs (m/s)_mean'] = weather_wide[['WVs (m/s)_x', 'WVs (m/s)_y', 'WVs (m/s)']].mean(axis=1)
weather_wide['WVv (m/s)_mean'] = weather_wide[['WVv (m/s)_x', 'WVv (m/s)_y', 'WVv (m/s)']].mean(axis=1)
weather_wide['p (hPa)_mean'] = weather_wide[['p (hPa)_x', 'p (hPa)_y', 'p (hPa)']].mean(axis=1)

# as we are only interested in the mean of the weather variables, we can now drop the specific weather variables of each station
weather_wide = weather_wide.drop(columns=['index', 'Hr (%Hr)_x', 'Hr (%Hr)_y', 'Hr (%Hr)', 'RainDur (min)_x', 'RainDur (min)_y', 'RainDur (min)', 'StrGlo (W/m2)_x', 'StrGlo (W/m2)_y', 'StrGlo (W/m2)', 'Temp (°C)_x', 'Temp (°C)_y', 'Temp (°C)', 'WD (°)_x', 'WD (°)_y', 'WD (°)', 'WVs (m/s)_x', 'WVs (m/s)_y', 'WVs (m/s)', 'WVv (m/s)_x', 'WVv (m/s)_y', 'WVv (m/s)', 'p (hPa)_x', 'p (hPa)_y', 'p (hPa)', 'Standort_x', 'Standort_y', 'Standort'])

In [ ]:
weather_wide.head(3)

#### 1.5.3 Weather: Data Inspection

In [ ]:
np.where(weather_wide['StrGlo (W/m2)_mean'].isna() == True) # did this code cell after knowing about the NaN's (see bellow). We wanted to know
# how the 143 missing values are distributed in the dataframe. Result: we have maximum 27h hours in series NaN's, followed by 21h and 9h in a
# a row. This result gives information wether the imputation methode of the NaN's is acceptable or not.

In [ ]:
weather_wide.info()

The columns 'Hr (%Hr)_mean', 'StrGlo (W/m2)_mean', 'Temp (°C)_mean', 'WVs (m/s)_mean', contain missing values

In [ ]:
# we will drop the column WVs (m/s)_mean as we have to many missing values to select a representiv formula to impute the missing values
weather_wide.drop('WVs (m/s)_mean',axis=1, inplace=True)

#### 1.5.4 Weather: Creating Time Shift

In [ ]:
# removing the UTC +1 timezone information
weather_wide['DateTime'] = pd.to_datetime(weather_wide['DateTime']).dt.tz_convert(None)

In [ ]:
# Now we want to add the value of each weather variable from the n-1st, n-2nd and n-3rd hour to the row of the n-th hour.

# List of Wettervariablen
wettervariablen = ['Hr (%Hr)_mean', 'RainDur (min)_mean', 'StrGlo (W/m2)_mean', 'Temp (°C)_mean', 'WD (°)_mean', 'p (hPa)_mean', 'WVv (m/s)_mean']

weather_wide.sort_values(by='DateTime', inplace=True)

# Iteriere über jede Wettervariable und füge die verschobenen Werte (um eine Stunde) hinzu
for variable in wettervariablen:
    weather_wide.loc[:, f'{variable}-1h'] = weather_wide.loc[:, variable].shift(1)
# Iteriere über jede Wettervariable und füge die verschobenen Werte (um zwei Stunde) hinzu
    weather_wide.loc[:, f'{variable}-2h'] = weather_wide.loc[:, variable].shift(2)
# Iteriere über jede Wettervariable und füge die verschobenen Werte (um drei Stunde) hinzu
    weather_wide.loc[:, f'{variable}-3h'] = weather_wide.loc[:, variable].shift(3)
    
weather_wide.drop(columns=wettervariablen, inplace=True) # drop the original columns to prevent data leakage

In [ ]:
weather_wide.head(3)

In [ ]:
# storing the exact dates of the daylightssavings time shift in a dictionary
timeshiftdic = {
    2012: [datetime(2012, 3, 25, 2, 0, 0), datetime(2012, 10, 28, 2, 0, 0)],
    2013: [datetime(2013, 3, 31, 2, 0, 0), datetime(2013, 10, 27, 2, 0, 0)],
    2014: [datetime(2014, 3, 30, 2, 0, 0), datetime(2014, 10, 26, 2, 0, 0)],
    2015: [datetime(2015, 3, 29, 2, 0, 0), datetime(2015, 10, 25, 2, 0, 0)],
    2016: [datetime(2016, 3, 27, 2, 0, 0), datetime(2016, 10, 30, 2, 0, 0)],
    2017: [datetime(2017, 3, 26, 2, 0, 0), datetime(2017, 10, 29, 2, 0, 0)],
    2018: [datetime(2018, 3, 25, 2, 0, 0), datetime(2018, 10, 28, 2, 0, 0)],
    2019: [datetime(2019, 3, 31, 2, 0, 0), datetime(2019, 10, 27, 2, 0, 0)],
    2020: [datetime(2020, 3, 29, 2, 0, 0), datetime(2020, 10, 25, 2, 0, 0)],
    2021: [datetime(2021, 3, 28, 2, 0, 0), datetime(2021, 10, 31, 2, 0, 0)],
    2022: [datetime(2022, 3, 27, 2, 0, 0), datetime(2022, 10, 30, 2, 0, 0)]
}

In [ ]:
# iterate over all years and shift the DateTime column by one hour for the daylight savings time
for year in tqdm(timeshiftdic.keys()):
    # delete the hour that would be duplicated by the shift
    weather_wide.drop(weather_wide.loc[weather_wide['DateTime'] == timeshiftdic[year][1]].index, inplace=True)
    weather_wide.loc[(weather_wide['DateTime'] >= timeshiftdic[year][0]) & (weather_wide['DateTime'] < timeshiftdic[year][1]), 'DateTime'] += timedelta(hours=1)

#### 1.5.5 Weather: Data Imputation

We found out, that in column 'Hr (%Hr)_mean', 'StrGlo (W/m2)_mean', 'Temp (°C)_mean' we have missing values. 
Therefore we replace these NaNs with the previous value of the weather_wide df (i.e. we replace the missing values with the values 
from the previous hour). Since the weather varies from day to day over the years, we find it most appropriate to use the previous hour's value.

In [ ]:
# replace NaN-Values in the column 'Hr (%Hr)_mean' by the previous value
weather_wide.ffill(inplace=True) # first forward fill all NaN's

In [ ]:
# controll check if there are no more missing values
# first we remove 2011
weather_wide = weather_wide[weather_wide['DateTime'].dt.year != 2011]
# then we check if there are any missing values
weather_wide.isna().value_counts()

#### 1.5.6 Weather: Merging weather_wide with accidents_raw

In [ ]:
# Now, perform the merge
accidents_raw = pd.merge(accidents_raw, weather_wide, how='left', on='DateTime')

In [ ]:
accidents_raw.head(3)

In [ ]:
weather_coords = {'E': [2681943, 2683148, 2682106], 'N': [1247245, 1249020, 1249935]}

fig, ax = plt.subplots(figsize=(30, 30))
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')
ax.scatter(weather_coords['E'], weather_coords['N'], s=150, alpha=0.8, color='green', marker='p')

### 1.6 Feature Engineering: Public Transport Stops and Garages

In [ ]:
# Read the files we're gonna use
haltestellen = pd.read_csv('Datasets/Haltestellen.csv', usecols=['SYMB_TEXT', 'E', 'N']) # shows all busstop's in the City of Zürich
haltestellen_coords = haltestellen[['E', 'N']]

garage = pd.read_csv('Datasets/stzh.poi_parkhaus_view.csv', usecols=['anzahl_oeffentliche_pp', 'geometry'])
garage['geometry'] = garage['geometry'].apply(shp.wkt.loads)
garage = gpd.GeoDataFrame(garage, geometry='geometry')

garage_coords = garage[['geometry']].copy()

In [ ]:
haltestellen.head(3)

In [ ]:
garage.head(3)

In [ ]:
haltestellen['SYMB_TEXT'].value_counts()

To create features for the type of public transport stop, we take the SYMB_TEXT column and create boolean values, based on wether a given stop contains a certain vehicle in its string.

In [ ]:
haltestellen['Bus'] = haltestellen['SYMB_TEXT'].str.contains('Bus')
haltestellen['Tram'] = haltestellen['SYMB_TEXT'].str.contains('Tram')
haltestellen['Bahn'] = haltestellen['SYMB_TEXT'].str.contains('Bahn')
haltestellen['Bergbahn'] = haltestellen['SYMB_TEXT'].str.contains('Bergbahn')
haltestellen.drop(columns=['SYMB_TEXT'], inplace=True)

Some stops dont contain a value for SYMB_TEXT. These are mainly fairy stops at lake Zurich. We decided to leave them in and give the model the opportunity to find certain patterns relating to stops, where all vehicle booleans are False.

In [ ]:
haltestellen.fillna(False, inplace=True)

In [ ]:
haltestellen.head(3)

In [ ]:
garage['anzahl_oeffentliche_pp'].isna().sum()

Creating a GeoDataFrame for spacial indexing.

In [ ]:
haltestellen['geometry'] = [shp.Point(x, y) for x, y in zip(haltestellen['E'], haltestellen['N'])]
haltestellen = gpd.GeoDataFrame(haltestellen, geometry='geometry').drop(['E', 'N'], axis=1)

In [ ]:
for indexU, accident_point in enumerate(tqdm(accidents_raw['geometry'])):
    
    # Find and assign closest parking garage and public transport stop
    indexH = haltestellen.sindex.nearest(geometry=accident_point)[1][0]
    accidents_raw.loc[indexU, 'PTS_id'] = indexH
    accidents_raw.loc[indexU, 'distance_PTS'] = math.dist(accident_point.coords[0], haltestellen.loc[indexH, 'geometry'].coords[0])
    indexG = garage.sindex.nearest(geometry=accident_point)[1][0]
    accidents_raw.loc[indexU, 'garage_id'] = indexG
    accidents_raw.loc[indexU, 'distance_garage'] = math.dist(accident_point.coords[0], haltestellen.loc[indexG, 'geometry'].coords[0])    
    
haltestellen.drop(columns=['geometry'], inplace=True)
garage.drop(columns=['geometry'], inplace=True)
accidents_raw = pd.merge(accidents_raw, haltestellen, how='left', left_on='PTS_id', right_index=True)
accidents_raw = pd.merge(accidents_raw, garage, how='left', left_on='garage_id', right_index=True)

In [ ]:
accidents_raw.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')
ax.scatter(weather_coords['E'], weather_coords['N'], s=150, alpha=0.8, color='green', marker='p')
ax.scatter(haltestellen_coords['E'], haltestellen_coords['N'], alpha=0.5, s=60, color='orange', marker='v')
garage_coords['geometry'].plot(ax=ax, alpha=0.5, markersize=100, color='grey', marker='s')

### 1.7 Feature Engineering: Traffic Zones

This dataset contains the geometries and type of all traffic zones in Zurich City.

In [ ]:
zonen = gpd.read_file('Datasets/Verkehrszonen.gpkg', geometry='geometry')[['zonentyp_technical', 'umgesetzt_datum', 'geometry']]

In [ ]:
zonen.head(3)

The code below checks for invalid geometries and finds one at index 126. This issue is then resolved by using the shapely buffer function.

In [ ]:
for i in range(307):
    if shp.is_valid_reason(zonen['geometry'][i]) != 'Valid Geometry':
        print(i, shp.is_valid_reason(zonen['geometry'][i]))

In [ ]:
zonen['geometry'][126] = shp.buffer(zonen['geometry'][126], 0)

In [ ]:
accidents_raw['DateTime']

Now each zone is iterated and checked for which accidents happened inside it and wether it was already instated at the time of the accident.

In [ ]:
for index, polygon in enumerate(tqdm(zonen['geometry'])):
    # Erster Schritt: Verwenden von 'contains' für räumliche Überprüfung
    spatial_mask = shp.contains(polygon, accidents_raw['geometry'])

    # Zweiter Schritt: Überprüfen des Datums, bei Nan-Werten wird die Bedingung automatisch erfüllt
    date_condition = (accidents_raw.loc[spatial_mask, 'DateTime'] >= zonen['umgesetzt_datum'][index]) | pd.isna(zonen['umgesetzt_datum'][index])
    
    # Kombinieren Sie räumliche und zeitliche Bedingungen
    mask = spatial_mask & date_condition
    #mask = shp.contains(polygon, accidents_raw['geometry']) and accidents_raw['date'] >= zonen['umgesetzt_datum'][zonen['geometry'] == polygon].values[0]
    accidents_raw.loc[mask, 'zone'] = zonen['zonentyp_technical'][zonen['geometry'] == polygon].values[0]
    accidents_raw.loc[mask, 'zonenid'] = zonen[zonen['geometry'] == polygon].index[0]
    
accidents_raw['zone'] = accidents_raw['zone'].fillna('no_zone')
accidents_raw['zonenid'] = accidents_raw['zonenid'].fillna('no_zoneid')

In [ ]:
zonencolors = {'T0': '#802b00', 'T20': '#b36b00', 'T30': '#ffcc00'}
zonen['color'] = zonen['zonentyp_technical'].map(zonencolors)
zonen_coords = zonen[['geometry', 'color']].copy()
zonen.drop(['color'], axis=1, inplace=True)

fig, ax = plt.subplots(figsize=(30, 30))
zonen_coords["geometry"].plot(facecolor=zonen_coords['color'], ax=ax, alpha=0.5)
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')
ax.scatter(weather_coords['E'], weather_coords['N'], s=150, alpha=0.8, color='green', marker='p')
garage_coords['geometry'].plot(ax=ax, alpha=0.5, markersize=100, color='grey', marker='s')
ax.scatter(haltestellen_coords['E'], haltestellen_coords['N'], alpha=0.5, s=60, color='orange', marker='v')

### 1.8 Feature Engineering: Street Network with Speedlimit

This dataset allows us to assign a street and its speedlimit to each accident, as well as calculate the distance to the next intersection and pedestrian crossings.

In [ ]:
strassen = pd.read_csv('Datasets/Strassen_mit_Geschwindigkeit.csv')[['objectid', 'temporegime_technical', 'umgesetzt_datum', 'geometry']]
strassen['geometry'] = strassen['geometry'].apply(shp.wkt.loads)
strassen = gpd.GeoDataFrame(strassen, geometry='geometry')
strassen['umgesetzt_datum'] = pd.to_datetime(strassen['umgesetzt_datum'], format='%Y%m%d%H%M%S%f')

In [ ]:
strassen.head(3)

In [ ]:
strassen.loc[~strassen.umgesetzt_datum.isna(), 'temporegime_technical'].value_counts()

This dataset also contains some invalid geometries, that need to be resolved first.

In [ ]:
for i in range(len(strassen)):
    if shp.is_valid_reason(strassen['geometry'][i]) != 'Valid Geometry':
        print(i, shp.is_valid_reason(strassen['geometry'][i]))

In [ ]:
strassen = strassen.drop(2692).reset_index(drop=True)

In [ ]:
for i in range(len(strassen)):
    if strassen['geometry'][i].is_empty:
        print(i, strassen['geometry'][i])

In [ ]:
for i in range(len(strassen)):
    if strassen['geometry'][i].geom_type != 'LineString':
        print(i, strassen['geometry'][i].geom_type)

In [ ]:
strassen = strassen.explode(index_parts=True).reset_index(drop=True)

#### 1.8.1 Streets: Speedlimit

As the following graph illustrates, spacial indexing allows us to quickly find the closest street geometry for each accident point and assign the closest one (orange line in the picture)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(accidents_raw['geometry'][1000].x, accidents_raw['geometry'][1000].y, color='green', s=20)
strassen["geometry"][100:120].plot(ax=ax, color='blue')
for i in range(21):
    gpd.GeoSeries(shp.shortest_line(accidents_raw['geometry'][1000], strassen['geometry'][i+100])).plot(ax=ax, color='red')
actual_nearest_index = strassen.sindex.nearest(geometry=accidents_raw['geometry'][1000])[1][0]
line_gdf = gpd.GeoDataFrame(geometry=[strassen['geometry'].iloc[actual_nearest_index]])
line_gdf.plot(ax=ax, color='orange')

In [ ]:
#für jeden accidents_raw
for indexU, accidents_rawpunkt in enumerate(tqdm(accidents_raw['geometry'])):
    #umsetzungsdatum der Strassen erstmal ignoriert
    #Nächste Strassen durch spatial index finden
    indexS = strassen.sindex.nearest(geometry=accidents_rawpunkt)[1][0]
    accidents_raw.loc[indexU, 'streetid'] = indexS
    accidents_raw.loc[indexU, 'speedlimit'] = strassen.loc[indexS, 'temporegime_technical']

accidents_raw['speedlimit'] = accidents_raw['speedlimit'].str[1:3].astype(int)

In [ ]:
accidents_raw.head(3)

In [ ]:
colorsstreets = {'T0': '#262626', 'T20': '#590d59', 'T30': '#821782', 'T50': '#862d59', 'T60': '#990033', 'T80': '#993333', 'T100': '#8a0f0f', 'T120': '#e63900', 'T50N30': '#862d59', 'T30N0': '#821782'}
strassen['color'] = strassen['temporegime_technical'].map(colorsstreets)
strassen['color'].fillna('#3e3e5b', inplace=True)
strassen_coords = strassen[['geometry', 'color']].copy()
strassen.drop(columns=['color'], inplace=True)

fig, ax = plt.subplots(figsize=(30, 30))
zonen_coords['geometry'].plot(facecolor=zonen_coords['color'], ax=ax, alpha=0.5)
strassen_coords['geometry'].plot(ax=ax, color=strassen_coords['color'], linewidth=1, alpha=0.8)
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')
ax.scatter(weather_coords['E'], weather_coords['N'], s=150, alpha=0.8, color='green', marker='p')
garage_coords['geometry'].plot(ax=ax, alpha=0.5, markersize=100, color='grey', marker='s')
ax.scatter(haltestellen_coords['E'], haltestellen_coords['N'], alpha=0.5, s=60, color='orange', marker='v')

#### 1.8.2 Streets: Intersections

Again we want to illustrate our approach. As the following graph shows, each intersection is represented as a point in the linestring geometries of at least two different streets. 

In [ ]:
a = strassen[strassen['objectid']==3593]['geometry']
b = strassen['geometry'][5164]
fig, ax = plt.subplots(figsize=(10,10))
a.plot(ax=ax)
for i in range(len(b.coords)):
    plt.scatter(b.coords[i][0], b.coords[i][1], c='red', s=50)
a2 = strassen[strassen['objectid']==4434]['geometry']
b2 = strassen['geometry'][5179]
a2.plot(ax=ax)
for i in range(len(b2.coords)):
    plt.scatter(b2.coords[i][0], b2.coords[i][1], c='red', s=50)

In [ ]:
koordinaten = [(index, coord) for index, geometry in enumerate(strassen['geometry']) for coord in geometry.coords]
koordinaten = list(set(koordinaten)) # remove duplicates (same point on same linestring)
counter = Counter(coord for index, coord in koordinaten)
koordinaten = [entry for entry in koordinaten if counter[entry[1]] > 1]
potentielle_kreuzungen = {coord: [index for index, c in koordinaten if c == coord] for coord, count in tqdm(counter.items()) if count > 1}

Now that we found all points that are present in at least two linestrings, we need to filter those out that are just a continuation of the street, so neither is there a dead end at the point, nor is there an intersection.

In [ ]:
weiterleitung = {}
kreuzungen = potentielle_kreuzungen.copy()

for coord, indices in tqdm(potentielle_kreuzungen.items()):
    if len(indices) == 2:
        link = True
        for i in range(2):
            for j in range(len(strassen['geometry'][indices[i]].coords)):
                if j > 0 and j < len(strassen['geometry'][indices[i]].coords) -1:
                    if strassen['geometry'][indices[i]].coords[j] == coord:
                        link = False
                        break
        
        if link:
            weiterleitung[coord] = indices
            kreuzungen.pop(coord, None)

In [ ]:
len(kreuzungen) + len(weiterleitung) == len(potentielle_kreuzungen)

The following code calculates the distance of the accident to the next intersection in both ways of the street network. It does this by first utilizing the shortest_line function and assigning each accident point its closest point on the street network. It then 'travels' the street in both directions by jumping from point to point in the linestrings of the street, adding up the distance travelled. At each point it checks wether it is the dictionary wit all intersections. If it does not find an intersection and reaches one end of the street, it checks wether the last point is in the continuation dictionary and continues the search on the next street, otherwise it found a dead end.

In [ ]:
def find_intersection_left(point, linestring, lineindex, streetindex):
    d = 0
    i = -1
        
    for k in range(lineindex-1, -1, -1): # eine richtung 
        d += math.dist(linestring.coords[k], linestring.coords[k+1])
        
        if linestring.coords[k] in kreuzungen:
            i = list(kreuzungen.keys()).index(linestring.coords[k])
            break
        
    if i == -1: # no intersection found
        point = linestring.coords[0]
        if point in weiterleitung:
            
            if weiterleitung[point][0] != streetindex: # taking the other street at the intersection, not the one we came from
                streetindex2 = weiterleitung[point][0]
                
            elif weiterleitung[point][0] == streetindex:
                streetindex2 = weiterleitung[point][1]
                
            if point == strassen['geometry'][streetindex2].coords[0]: # if the link point is at the beginning of the other linestring
                d2, i = find_intersection_right(point, strassen['geometry'][streetindex2], 0, streetindex2)
                d += d2
                
            elif point == strassen['geometry'][streetindex2].coords[len(strassen['geometry'][streetindex2].coords)-1]: # if the link point is at the end of the other linestring
                d2, i = find_intersection_left(point, strassen['geometry'][streetindex2], len(strassen['geometry'][streetindex2].coords)-1, streetindex2)
                d += d2
                    
        # else: dead end, take distance to last point, do nothing, i = -1 means dead end
    
    return d, i
        
def find_intersection_right(point, linestring, lineindex, streetindex):
    d = 0
    i = -1
         
    for l in range(lineindex + 1, len(linestring.coords), 1): # eine richtung 
        d += math.dist(linestring.coords[l], linestring.coords[l-1])
        
        if linestring.coords[l] in kreuzungen:
            i = list(kreuzungen.keys()).index(linestring.coords[l])
            break
        
    if i == -1: # no intersection found
        point = linestring.coords[len(linestring.coords)-1]
        if point in weiterleitung:
            
            if weiterleitung[point][0] != streetindex: # taking the other street at the intersection, not the one we came from
                streetindex2 = weiterleitung[point][0]
            else:
                streetindex2 = weiterleitung[point][1]
                
            if point == strassen['geometry'][streetindex2].coords[0]: # if the link point is at the beginning of the other linestring
                d, i = find_intersection_right(point, strassen['geometry'][streetindex2], 0, streetindex2)
                
            elif point == strassen['geometry'][streetindex2].coords[len(strassen['geometry'][streetindex2].coords)-1]: # if the link point is at the end of the other linestring
                d, i = find_intersection_left(point, strassen['geometry'][streetindex2], len(strassen['geometry'][streetindex2].coords)-1, streetindex2)
                    
        # else: dead end, take distance to last point, do nothing, i = -1 means dead end
    
    return d, i

# für jeden row
# def kreuzungssuche(row):
for i, accident_point in enumerate(tqdm(accidents_raw['geometry'])):
    d1 = 0
    d2 = 0
    ds = 0
    i1 = -1
    i2 = -1
    
    streetindex = accidents_raw['streetid'][i]
    linestring = strassen['geometry'][streetindex]
    point_on_linestring = shp.shortest_line(accident_point, linestring).coords[1]

    for j in range(len(linestring.coords)-1): # iterate over linestring to find position of point_on_linestring
        if point_on_linestring == linestring.coords[j]: # Point already in linestring
            if linestring.coords[j] in kreuzungen: 
                i1 = list(kreuzungen.keys()).index(linestring.coords[j])
                i2 = i1
                break
            else: # Point not in kreuzungen
                d1, i1 = find_intersection_left(point_on_linestring, linestring, j, streetindex)
                d2, i2 = find_intersection_right(point_on_linestring, linestring, j, streetindex)
                break
            
        elif shp.Point(point_on_linestring).within(shp.LineString([linestring.coords[j], linestring.coords[j+1]]).buffer(0.0000001)):  # point between two points of linestring
            new_linestring = shp.LineString([point for point in linestring.coords[:j+1] + [point_on_linestring] + linestring.coords[j+1:]])
            d1, i1 = find_intersection_left(point_on_linestring, new_linestring, j+1, streetindex)
            d2, i2 = find_intersection_right(point_on_linestring, new_linestring, j+1, streetindex)
            break
    
    ds = math.dist(accident_point.coords[0], point_on_linestring)    
    d1 += ds
    d2 += ds
        
    accidents_raw.loc[i, 'distance_intersection_1'] = d1
    accidents_raw.loc[i, 'id_intersection_1'] = i1
    accidents_raw.loc[i, 'distance_intersection_2'] = d2
    accidents_raw.loc[i, 'id_intersection_2'] = i2
    accidents_raw.loc[i, 'distance_to_street'] = ds

In [ ]:
accidents_raw['closest_intersection_id'] = accidents_raw[['id_intersection_1', 'id_intersection_2']].min(axis=1)

In [ ]:
accidents_raw.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
xmin, xmax = 2680500, 2685500
ymin, ymax = 1245600, 1250600
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
zonen_coords['geometry'].plot(facecolor=zonen_coords['color'], ax=ax, alpha=0.5)
strassen_coords['geometry'].plot(ax=ax, color=strassen_coords['color'], linewidth=1, alpha=0.8)
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')
ax.scatter(weather_coords['E'], weather_coords['N'], s=150, alpha=0.8, color='green', marker='p')
ax.scatter(*zip(*weiterleitung.keys()), c='#00cc44', s=30, alpha=0.6)
ax.scatter(*zip(*kreuzungen.keys()), c='#0033cc', s=30, alpha=0.6, marker='x')
garage_coords['geometry'].plot(ax=ax, alpha=0.5, markersize=100, color='grey', marker='s')
ax.scatter(haltestellen_coords['E'], haltestellen_coords['N'], alpha=0.5, s=60, color='orange', marker='v')

### 1.9 Feature Engineering: Pedestrian and Bicycle Network

This dataset helps us find pedestrian crossings and types of bicycle paths that might be helpful to our model for predicting personal injuries that are probably especially likely for accidents involving pedestrians or bicycles.

In [ ]:
fvnetz = gpd.read_file('Datasets/Fuss-_und_Velonetz.gpkg', geometry='geometry')[['velo', 'velostreifen', 'veloweg', 'fuss', 'geometry']]
fvnetz.head(3)

In [ ]:
fvnetz['velostreifen'].value_counts()

In [ ]:
fvnetz['velostreifen'] = fvnetz['velostreifen'].map(lambda x: 1 if x in ['BOTH', 'FT', 'TF', '1'] else 0) # we disregard one-way paths and just count them as true

In [ ]:
for i in range(len(fvnetz)):
    if fvnetz['geometry'][i].is_empty:
        print(i, fvnetz['geometry'][i].geom_type)

In [ ]:
for i in range(len(fvnetz)):
    if fvnetz['geometry'][i].geom_type != 'LineString':
        print(i, fvnetz['geometry'][i].geom_type)

#### 1.9.1 Finding Pedestrian Crossings

The following code searches for all pedestrain crossings, by checking wether their geometries cross.

In [ ]:
lines1 = fvnetz.loc[fvnetz.fuss == 1, 'geometry']
lines2 = strassen['geometry'].sindex
crossings = {}

for line1 in tqdm(lines1):
    potential_crossings = list(lines2.intersection(line1.bounds))
    
    for i in potential_crossings:
        line2 = strassen.iloc[i]['geometry']
        if strassen.iloc[i]['temporegime_technical'] == 'T0':
            continue # This is used to filter out all crossings on streets were driving is prohibited, because they themselves are mostly paths for pedestrian use
        # Check if the LineStrings cross
        if line1.crosses(line2):
            
            # Find the crossing points and append to the list
            intersection = line1.intersection(line2)       
            if intersection.geom_type == 'Point':
                #rounded_point = (intersection.x.round(2), intersection.y.round(2))  
                crossings[intersection] = i
            elif intersection.geom_type == 'MultiPoint':
                for p in gpd.GeoSeries(intersection).explode(index_parts=True):
                    #rounded_point = (p.x.round(2), p.y.round(2))  
                    crossings[p] = i
                


Checking wether we calculated the crossings correctly, by plotting exemplary geometries

In [ ]:
x = [x for x in strassen.loc[4618, 'geometry'].xy[0]]
y = [y for y in strassen.loc[4618, 'geometry'].xy[1]]
plt.plot(x, y, c='blue')
plt.plot(2683457.176, 1248857.039, c='red', marker='o')

In [ ]:
velo = fvnetz.loc[fvnetz.velo == 1].reset_index(drop=True)
accidents_raw['is_crossing_on_street'] = False
accidents_raw['is_crossing_the_same'] = False
crossing_keys = list(crossings.keys())
crossing_values = list(crossings.values())
geo_crossing_keys = gpd.GeoSeries(crossing_keys)

for i, accident_point in enumerate(tqdm(accidents_raw['geometry'])):
    #Nächste Strassen durch spatial index finden
    sindex = velo.sindex.nearest(geometry=accident_point)[1][0]
    accidents_raw.loc[i, 'velo_id'] = sindex
    accidents_raw.loc[i, 'velo_distance'] = shp.distance(accident_point, velo.loc[sindex, 'geometry'])
    accidents_raw.loc[i, 'veloweg'] = velo.loc[sindex, 'veloweg']
    accidents_raw.loc[i, 'velostreifen'] = velo.loc[sindex, 'velostreifen']

    crossingindex = geo_crossing_keys.sindex.nearest(geometry=accident_point)[1][0]
    crossing = crossing_keys[crossingindex]
    onstreet = accidents_raw['streetid'][i] in crossing_values
    thesame = crossing_values[crossingindex] == accidents_raw['streetid'][i]
    
    accidents_raw.loc[i, 'distance_to_closest_crossing'] = shp.distance(accident_point, crossing)
    accidents_raw.loc[i, 'closest_crossing_id'] = crossingindex
    accidents_raw.loc[i, 'is_crossing_on_street'] = onstreet
    accidents_raw.loc[i, 'is_crossing_the_same'] = thesame

In [ ]:
accidents_raw.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
xmin, xmax = 2680500, 2685500
ymin, ymax = 1245600, 1250600
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
zonen_coords['geometry'].plot(facecolor=zonen_coords['color'], ax=ax, alpha=0.5)
strassen_coords['geometry'].plot(ax=ax, color=strassen_coords['color'], linewidth=1, alpha=0.8)
accidents_raw['geometry'].plot(ax=ax, alpha=0.2, markersize=4)
ax.scatter(traffic_coords['EKoord'], traffic_coords['NKoord'], s=50, color='#4d004d', alpha=0.8, marker='D')
ax.scatter(weather_coords['E'], weather_coords['N'], s=150, alpha=0.8, color='green', marker='p')
ax.scatter(*zip(*weiterleitung.keys()), c='#00cc44', s=30, alpha=0.6)
ax.scatter(*zip(*kreuzungen.keys()), c='#0033cc', s=30, alpha=0.6, marker='x')
fvnetz.loc[fvnetz.fuss == 1, 'geometry'].plot(ax=ax, color='#996633', linewidth=0.3)
fvnetz.loc[fvnetz.velostreifen == 1, 'geometry'].plot(ax=ax, color='#00e6b8', linewidth=0.5)
fvnetz.loc[fvnetz.veloweg == 1, 'geometry'].plot(ax=ax, color='#004d00', linewidth=0.5)
ax.scatter([point.x for point in crossings.keys()], [point.y for point in crossings.keys()], c='#800000', s=20, alpha=0.4)
garage_coords['geometry'].plot(ax=ax, alpha=0.5, markersize=100, color='grey', marker='s')
ax.scatter(haltestellen_coords['E'], haltestellen_coords['N'], alpha=0.5, s=60, color='orange', marker='v')

### 1.10 Holidays in Zurich City

Finaly we ad a boolean column for the holidays in Zurich City. This includes publicly ackknowledged holidays on the federal, cantonal and local level.

In [ ]:
holidays = []
sechseläutendic = {
    2012: date(2012, 4, 16),
    2013: date(2013, 4, 15),
    2014: date(2014, 4, 28),
    2015: date(2015, 4, 13),
    2016: date(2016, 4, 18),
    2017: date(2017, 4, 24),
    2018: date(2018, 4, 16),
    2019: date(2019, 4, 8),
    2020: date(2020, 4, 20),
    2021: date(2021, 4, 19),
    2022: date(2022, 4, 25),
}

knabenschiessendic = {
    2012: date(2012, 9, 10),
    2013: date(2013, 9, 9),
    2014: date(2014, 9, 15),
    2015: date(2015, 9, 14),
    2016: date(2016, 9, 12),
    2017: date(2017, 9, 11),
    2018: date(2018, 9, 10),
    2019: date(2019, 9, 9),
    2020: None,
    2021: date(2021, 9, 13),
    2022: date(2022, 9, 12),
}

for year in range(2012, 2023):
    vortag_karfreitag = easter(year) - timedelta(days=3)
    karfreitag = easter(year) - timedelta(days=2)
    easterday = easter(year)
    ostermontag = easter(year) + timedelta(days=1)
    vortag_auffahrt = easter(year) + timedelta(days=38)
    auffahrt = easter(year) + timedelta(days=39)  # Ascension Day is 39 days after Easter
    pfingstmontag = easter(year) + timedelta(days=49)  # Whit Monday is 49 days after Easter
    sechseläuten = sechseläutendic[year]
    knabenschiessen = knabenschiessendic[year]
    
    holidays.extend([
        vortag_karfreitag,
        karfreitag,
        easterday,
        ostermontag,
        vortag_auffahrt,
        auffahrt,
        pfingstmontag,
        sechseläuten,
        knabenschiessen,
        date(year, 2, 1),  # Berchtolds Day
        date(year, 1, 1),  # New Year's Day
        date(year, 1, 2),  # Second Day of New Year
        date(year, 5, 1),  # Labor Day
        date(year, 8, 1),  # Swiss National Day
        date(year, 12, 25),  # Christmas Day
        date(year, 12, 26),  # St. Stephen's Day
    ])

accidents_raw['is_holiday'] = accidents_raw['DateTime'].dt.date.isin(holidays)

In [ ]:
parks = gpd.read_file('Datasets/Parks.gpkg')[[('geometry')]]

In [ ]:
parks

### 1.11 Final Preparations

In [ ]:
# Preparing our label. as1 to 3 means personal injury, as0 means only property damage
accidents_raw['Severity'] = accidents_raw['AccidentSeverityCategory'].replace({'as1':1, 'as2':1, 'as3': 1,'as4':0,}).astype(bool)

In [ ]:
# saving the final raw dataset
accidents_raw.to_csv('accidents_raw.csv', index=False)

In [ ]:
# raw dataset can be loaded back in at any time, without going through the preprocessing again
accidents_raw = pd.read_csv('accidents_raw.csv')

In [ ]:
accidents = pd.DataFrame(accidents_raw.drop(columns=['DateTime', 'Date', 'geometry', 'AccidentUID', 'AccidentLocation_CHLV95_E', 'AccidentLocation_CHLV95_N', 'Date', 'AccidentYear',
                                                      'velo_id', 'id_intersection_1', 'id_intersection_2', 'closest_crossing_id', 'streetid', 'closest_intersection_id']))

In [ ]:
accidents = pd.get_dummies(accidents, columns=['AccidentType', 'RoadType', 'zone', 'AccidentMonth', 'AccidentWeekDay', 'AccidentHour', 'CountingStation_MinDistance', 'CountingStation_2thMinDistance', 'CountingStation_3thMinDistance', 'PTS_id', 'garage_id', 'zonenid'])

In [ ]:
accidents = accidents.dropna()

In [ ]:
accidents

## 2. Data Analysis

In [ ]:
df = pd.read_csv("accidents_raw.csv")
df = df.dropna(subset=['CountingStation_MinDistance'])

### 2.1 Analysis by time of day

In [ ]:
accidents_per_hour = df.groupby(['AccidentHour', 'Severity']).size().unstack(fill_value=0).reset_index()
accidents_per_hour["sum"] = accidents_per_hour[0] + accidents_per_hour[1] # Sum of all accident types (0 is equal to 'Accident with property damage' and 1 is equal to 'Accident with personal injury')
accidents_per_hour["Accidents with property damage %"] = accidents_per_hour[0] / accidents_per_hour["sum"] * 100
accidents_per_hour["Accidents with personal injury %"] = accidents_per_hour[1] / accidents_per_hour["sum"] * 100
accidents_per_hour.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_per_hour.plot(
    x="AccidentHour",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accient Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel("Uhrzeit")
ax.set_ylabel("%")

### 2.2 Analysis by weekdays

In [ ]:
accidents_by_day = df.groupby(['AccidentWeekDay', 'Severity']).size().unstack(fill_value=0).reset_index()
accidents_by_day["sum"] = accidents_by_day[0] + accidents_by_day[1] # Sum of all accident types (0 is equal to 'Accident with property damage' and 1 is equal to 'Accident with personal injury')
accidents_by_day["Accidents with property damage %"] = accidents_by_day[0] / accidents_by_day["sum"] * 100
accidents_by_day["Accidents with personal injury %"] = accidents_by_day[1] / accidents_by_day["sum"] * 100

day_mapping = {
    1:"Monday",
    2:"Tuesday",
    3:"Wednesday",
    4:"Thursday",
    5:"Friday",
    6:"Saturday",
    7:"Sunday"
}
accidents_by_day['AccidentWeekDay_en'] = accidents_by_day['AccidentWeekDay'].map(day_mapping)
accidents_by_day.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_day.plot(
    x="AccidentWeekDay_en",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel("Weekday")
ax.set_ylabel("%")

### 2.3 Analysis by month

In [ ]:
accidents_by_month = df.groupby(['AccidentMonth', 'Severity']).size().unstack(fill_value=0).reset_index()

accidents_by_month["sum"] = accidents_by_month[0] + accidents_by_month[1]
accidents_by_month["Accidents with property damage %"] = accidents_by_month[0] / accidents_by_month["sum"] * 100
accidents_by_month["Accidents with personal injury %"] = accidents_by_month[1] / accidents_by_month["sum"] * 100


month_mapping = {
    1:"January",
    2:"February",
    3:"March",
    4:"April",
    5:"May",
    6:"June",
    7:"July",
    8:"August",
    9:"September",
    10:"October",
    11:"November",
    12:"December"
}

accidents_by_month['AccidentMonth_en'] = accidents_by_month['AccidentMonth'].map(month_mapping)
accidents_by_month.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_month.plot(
    x="AccidentMonth_en",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel("Month")
ax.set_ylabel("%")

### 2.4 Analysis by year

In [ ]:
accidents_by_year = df.groupby(['AccidentYear', 'Severity']).size().unstack(fill_value=0).reset_index()

accidents_by_year["sum"] = accidents_by_year[0] + accidents_by_year[1]
accidents_by_year["Accidents with property damage %"] = accidents_by_year[0] / accidents_by_year["sum"] * 100
accidents_by_year["Accidents with personal injury %"] = accidents_by_year[1] / accidents_by_year["sum"] * 100
accidents_by_year.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_year.plot(
    x="AccidentYear",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel("Year")
ax.set_ylabel("%")

### 2.5 Analysis by roadtype

In [ ]:
accidents_by_roadtype = df.groupby(['RoadType', 'Severity']).size().unstack(fill_value=0).reset_index()

accidents_by_roadtype["sum"] = accidents_by_roadtype[0] + accidents_by_roadtype[1]
accidents_by_roadtype["Accidents with property damage %"] = accidents_by_roadtype[0] / accidents_by_roadtype["sum"] * 100
accidents_by_roadtype["Accidents with personal injury %"] = accidents_by_roadtype[1] / accidents_by_roadtype["sum"] * 100


roadtype_mapping = {
    "rt432":"Principal road",
    "rt433":"Minor road",
    "rt439":"Other",
}

accidents_by_roadtype['RoadType_en'] = accidents_by_roadtype['RoadType'].map(roadtype_mapping)
accidents_by_roadtype.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_roadtype.plot(
    x="RoadType_en",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel("Road Type")
ax.set_ylabel("%")

### 2.6 Analysis by Accident Type

In [ ]:
accidents_by_accidenttype = df.groupby(['AccidentType', 'Severity']).size().unstack(fill_value=0).reset_index()
accidents_by_accidenttype["sum"] = accidents_by_accidenttype[0] + accidents_by_accidenttype[1]
accidents_by_accidenttype["Accidents with property damage %"] = accidents_by_accidenttype[0] / accidents_by_accidenttype["sum"] * 100
accidents_by_accidenttype["Accidents with personal injury %"] = accidents_by_accidenttype[1] / accidents_by_accidenttype["sum"] * 100

accidenttype_mapping = {
    "at0":"Accident with skidding or self-accident",
    "at00":"Other",
    "at1":"Accident when overtaking or changing lanes",
    "at2":"Accident with rear-end collision",
    "at3":"Accident when turning left or right",
    "at4":"Accident when turning-into main road",
    "at5":"Accident when crossing the lane(s)",
    "at6":"Accident with head-on collision",
    "at7":"Accident when parking",
    "at8":"Accident involving pedestrian(s)",
    "at9":"Accident involving animal(s)"
}

accidents_by_accidenttype['AccidentType_en'] = accidents_by_accidenttype['AccidentType'].map(accidenttype_mapping)
accidents_by_accidenttype.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_accidenttype.plot(
    x="AccidentType_en",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel("Accident Type")
ax.set_ylabel("%")

### 2.7 Analysis of whether motorcycle is involved

In [ ]:
accidents_by_motorcycle = df.groupby(['AccidentInvolvingMotorcycle', 'Severity']).size().unstack(fill_value=0).reset_index()
accidents_by_motorcycle["sum"] = accidents_by_motorcycle[0] + accidents_by_motorcycle[1]
accidents_by_motorcycle["Accidents with property damage %"] = accidents_by_motorcycle[0] / accidents_by_motorcycle["sum"] * 100
accidents_by_motorcycle["Accidents with personal injury %"] = accidents_by_motorcycle[1] / accidents_by_motorcycle["sum"] * 100
accidents_by_motorcycle.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_motorcycle.plot(
    x="AccidentInvolvingMotorcycle",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_ylabel("%")
ax.set_xlabel("Motorcycle involved")

### 2.7 Analysis by Next Traffic Frequency Station

In [ ]:
df_traffic = df.query("CountingStation_MinDistance in ['Kornhausbrücke Limmatplatz','Stauffacherstrasse (Feldstrasse) Seebahnstrasse','Stauffacherstrasse (Feldstrasse) Seebahnstrasse','Binzmühlestrasse (Thurgauerstrasse) Schwamendingen','Europabrücke (Max Högger- und Würzgrabenstrasse) Altstetten','Seebahnstrasse (Kalkbreitestrasse) Birmensdorferstrasse']")
accidents_by_countingstation_mindistance = df_traffic.groupby(['CountingStation_MinDistance', 'Severity']).size().unstack(fill_value=0).reset_index()
accidents_by_countingstation_mindistance["sum"] = accidents_by_countingstation_mindistance[0] + accidents_by_countingstation_mindistance[1]
accidents_by_countingstation_mindistance["Accidents with property damage %"] = accidents_by_countingstation_mindistance[0] / accidents_by_countingstation_mindistance["sum"] * 100
accidents_by_countingstation_mindistance["Accidents with personal injury %"] = accidents_by_countingstation_mindistance[1] / accidents_by_countingstation_mindistance["sum"] * 100
accidents_by_countingstation_mindistance.head(5)

In [ ]:
plt.figure(figsize=(12, 20))
ax = accidents_by_countingstation_mindistance.plot(
    x="CountingStation_MinDistance",
    y=["Accidents with personal injury %", "Accidents with property damage %"],
    kind="bar",
    color=['#AA4A44', '#20B2AA'],
    edgecolor='black',
    width=0.8
)

# To add space for the legend, we can use bbox_to_anchor to position the legend outside of the plot
ax.legend(title="Accident Category", bbox_to_anchor=(1, 1), loc='upper left')
ax.set_ylabel("%")
ax.set_xlabel("Traffic frequency counting stations")
ax.set_title("")

### 2.8 Analysis by Weather

In [ ]:
weather = df[["Severity","RainDur (min)_mean-1h","RainDur (min)_mean-2h",
              "StrGlo (W/m2)_mean-1h","StrGlo (W/m2)_mean-2h",
              "Temp (°C)_mean-1h","Temp (°C)_mean-2h",
              "WD (°)_mean-1h","WD (°)_mean-2h",
              "WVv (m/s)_mean-1h","WVv (m/s)_mean-2h",
              "p (hPa)_mean-1h","p (hPa)_mean-2h",
              "Hr (%Hr)_mean-1h","Hr (%Hr)_mean-2h"]]
weather.head(5)

In [ ]:
plt.figure(figsize=(30,20))
sns.heatmap(weather.corr(), annot=True, cmap="YlGnBu")
plt.title("Correlation matrix for the weather data set", fontsize=32, fontweight="bold")
plt.show()

### 2.9 Analysis of traffic volume

In [ ]:
traffic = df[["Severity",
            "CountingStation_MinDistance","MinDistance","AnzFahrzeuge_-1h","AnzFahrzeuge_-2h",
            "2thMinDistance","AnzFahrzeuge_2thMinDistance_-1h","AnzFahrzeuge_2thMinDistance_-2h",
            "3thMinDistance","AnzFahrzeuge_3thMinDistance_-1h","AnzFahrzeuge_3thMinDistance_-2h",]]
trafficfrequency = pd.DataFrame(traffic["CountingStation_MinDistance"].value_counts()).head(10).reset_index()
dummies = pd.get_dummies(trafficfrequency[['CountingStation_MinDistance']])
traffic = pd.concat([traffic, dummies], axis=1)
traffic.drop(columns=['CountingStation_MinDistance'],inplace=True)


In [ ]:
plt.figure(figsize=(30, 20))
sns.heatmap(traffic.corr(), annot=True, cmap="YlGnBu")
plt.title("Correlation matrix for the traffic data set for the 10 measuring stations with the most measurements", fontsize=24, fontweight="bold")
plt.show()

## 3. Model Training and comparison

In [ ]:
# Example data (replace this with your data)
X = accidents.drop(columns=['AccidentSeverityCategory', 'Severity'])
y = accidents[['AccidentSeverityCategory', 'Severity']]

# Split the data into training, validating and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=72)

z_train = y_train['AccidentSeverityCategory']
z_val = y_val['AccidentSeverityCategory']
z_test = y_test['AccidentSeverityCategory']

y_train = y_train['Severity']
y_val = y_val['Severity']
y_test = y_test['Severity']

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Anwenden von SMOTE auf den Trainingsdatensatz
smote = SMOTE(random_state=41)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay

def plot_results(y_probs, y_true=y_val):    
    
    desired_recall = 0.9 
    thresholds = np.arange(-0.2, 1, 0.005) 
    best_threshold = 0 
    best_roc_score = 0 
    recall_scores = [] 
    roc_scores = [] 
    
    for threshold in thresholds: 
        y_pred_val_custom_threshold = (y_probs > threshold).astype(int) 
        current_recall = recall_score(y_true, y_pred_val_custom_threshold) 
        
        if current_recall >= desired_recall: 
            current_roc_score = roc_auc_score(y_true, y_pred_val_custom_threshold)
            recall_scores.append(current_recall) 
            roc_scores.append(current_roc_score) 
    
    best_threshold_index = np.argmax(roc_scores) 
    best_threshold_value = thresholds[best_threshold_index] 

    best_roc_score = roc_scores[best_threshold_index] 
    
    cm = confusion_matrix(y_true, y_probs > best_threshold_value)

    # Plot ROC curve and confusion matrix side by side
    fig, ax = plt.subplots(1, 2, figsize=(12, 5)) 

    # Plot ROC curve and Recall scores
    ax[0].plot(thresholds[:len(recall_scores)], recall_scores, label='Recall')
    ax[0].plot(thresholds[:len(roc_scores)], roc_scores, label='ROC Score')
    ax[0].scatter([best_threshold_value], [best_roc_score], color='red', marker='o', label='Best Threshold')
    ax[0].set_xlabel('Threshold')
    ax[0].set_ylabel('Score')
    ax[0].legend()
    ax[0].set_title('Recall und ROC Score in Abhängigkeit vom Threshold')

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negative', 'Positive'])
    disp.plot(cmap='Blues', ax=ax[1])
    ax[1].set_title('Confusion Matrix at Threshold {:.3f}'.format(best_threshold))

    ## Add text with best threshold and ROC score above the plots
    text = f'Best Threshold: {best_threshold_value:.3f}, Best ROC Score: {best_roc_score:.3f}, Recall: {recall_score(y_true, y_probs > best_threshold_value):.3f}'
    fig.text(0.5, 1.05, text, ha='center', va='center', fontsize=12)

    plt.tight_layout()
    plt.show()


### 3.1 Random Forest

#### 3.1.1 Basic Random Forest Classifier

In [ ]:
rf_classifier = RandomForestClassifier()

# GridSearchCV-Objekt erstellen
rf_classifier.fit(X_train, y_train)

# Vorhersagen auf dem Validierungsset machen
y_prob_val_rf1 = rf_classifier.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_rf1)

#### 3.1.2 Hyperparamter tuning with GridSearchCV

First, we will optimize our model so that the ROC-AUC score is maximized, regardless of recall

In [ ]:
# Parametergitter für die Anzahl der Bäume
param_grid = {
    'criterion': ["gini","entropy"],
    'max_depth': [10,None],
    'min_samples_split': [2,5,7],
    'min_samples_leaf': [1,2,5]
}

# Random Forest Classifier erstellen
rf_classifier = RandomForestClassifier()

# GridSearchCV-Objekt erstellen
grid_search = GridSearchCV(rf_classifier, param_grid, cv=3)
grid_search.fit(X_train_resampled, y_train_resampled)

# Beste Parameter abrufen
best_params = grid_search.best_estimator_.get_params()
print("Best Parameters: ", best_params)

best_estimator = grid_search.best_estimator_

# Vorhersagen auf dem Validierungsset machen
y_prob_val_rf1 = best_estimator.predict_proba(X_val)[:, 1]

# Benutzerdefinierten Recall-Wert festlegen
desired_recall = 0.3  

# Finden Sie den Schwellenwert für den gewünschten Recall
thresholds = np.arange(0, 1.05, 0.05)
best_threshold = 0
best_roc_score = 0

recall_scores = []
roc_scores = []

for threshold in thresholds:
    y_pred_val_rf1_custom_threshold = (y_prob_val_rf1 > threshold).astype(int)
    current_recall = recall_score(y_val, y_pred_val_rf1_custom_threshold)
    
    if current_recall >= desired_recall:
        current_roc_score = roc_auc_score(y_val, y_pred_val_rf1_custom_threshold)
        
        recall_scores.append(current_recall)
        roc_scores.append(current_roc_score)


best_threshold_index = np.argmax(roc_scores)
best_threshold_value = thresholds[best_threshold_index]
best_roc_score = roc_scores[best_threshold_index]

In [ ]:
y_prob_val_rf1 = best_estimator.predict_proba(X_val)[:, 1]
y_pred_val_rf1 = (y_prob_val_rf1 > best_threshold_value).astype(int)

fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# First plot
axs[0].plot(thresholds[:len(recall_scores)], recall_scores, label='Recall')
axs[0].plot(thresholds[:len(roc_scores)], roc_scores, label='ROC Score')
axs[0].scatter([best_threshold_value], [best_roc_score], color='red', marker='o', label='Best Threshold')
axs[0].set_xlabel('Threshold')
axs[0].set_ylabel('Score')
axs[0].legend()
axs[0].set_title('Recall und ROC Score (validation set)')
axs[0].text(0.5, -0.15, f"Bester Threshold: {best_threshold_value:.2f}\nBester ROC Score: {best_roc_score:.2f}", 
            transform=axs[0].transAxes, horizontalalignment='center', verticalalignment='center', fontsize=16)


# Second plot
cm_val = confusion_matrix(y_val, y_pred_val_rf1)
sns.heatmap(cm_val, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'], ax=axs[1])
axs[1].set_title('Confusion matrix (validation set)')
axs[1].set_xlabel('Predicted Values')
axs[1].set_ylabel('Actual Values')


plt.subplots_adjust(wspace=0.5)
plt.tight_layout()
plt.show()

#### 3.1.3 Given a chosen recall, train a model that maximizes the ROC-AUC 

In [ ]:
# Parametergitter für die Anzahl der Bäume
param_grid = {
    'criterion': ["gini","entropy"],
    'max_depth': [10,None],
    'min_samples_split': [2,5,7],
    'min_samples_leaf': [1,2,5]
}

# Random Forest Classifier erstellen
rf_classifier = RandomForestClassifier()

# GridSearchCV-Objekt erstellen
grid_search = GridSearchCV(rf_classifier, param_grid, cv=3)
grid_search.fit(X_train_resampled, y_train_resampled)

# Beste Parameter abrufen
best_params = grid_search.best_estimator_.get_params()
print("Best Parameters: ", best_params)

best_estimator = grid_search.best_estimator_

# Vorhersagen auf dem Validierungsset machen
y_prob_val_rf2 = best_estimator.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_rf2)

+ As we have seen, there is a trade-off between maximizing precision and recall. We can't maximize both at the same time.
+ In our case, we would like to reduce the number of false negative results, as it can have fatal consequences if personal injury (a positive) is classified as property damage (a negative). We accept that this will increase the number of false positives. This meant: It would be better for an ambulance to drive to the scene of an accident in vain than for an injured person not to be offered the necessary outpatient help.
+ For this reason we choose a recall of 0.9 for the other models.
+ We also accept that a given recall of 0.9 may not give us the maximum possible value for the "Area Under the Curve" of the “Receiver Operating Characteristic” curve

#### 3.1.4 Hyperparamter tuning with RandomSearchCV

+ Although hyperparameter tuning with GridSearchCV is very accurate, this type of hyperparameter tuning requires a lot of computing power. For this reason, in a next step we will use Scikilearn's RandomizedSearchCV function.
+ The RandomizedSearchCV-function searches randomly selected combinations of the specified hyperparameter values. It performs a random search and only tries a set number of combinations.

In [ ]:
smote = SMOTE(random_state=41)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Parametergitter für die Anzahl der Bäume
param_grid = {
    'criterion': ["gini","entropy"],
    'max_depth': [10,20,30,None],
    'min_samples_split': [2,5,7,9],
    'min_samples_leaf': [2,5,7,9]
}

# Random Forest Classifier erstellen
rf_classifier = RandomForestClassifier()

# Weitere Optimierung der Hyperparameter
grid_search = RandomizedSearchCV(rf_classifier, param_grid, cv=3, n_iter=10, random_state=42)
grid_search.fit(X_train_resampled, y_train_resampled)

# Vorhersagen auf dem Validierungsset machen
y_prob_val_rf3 = grid_search.best_estimator_.predict_proba(X_val)[:, 1]
randomforestcv =  grid_search.best_estimator_

# Hier kommt die Funktion von Gabriel
plot_results(y_prob_val_rf3) 

+ Since the ROC-AUC score, which was calculated once by the model where we selected the hyperparamters with the GridSearchCV and once by the other model where we selected the hyperparamters with the RandomizedSearchCV function, does not differ greatly, we will use further the RandomizedSearchCV function below to find the optimal hyperparameters for the decision tree model.

### 3.2 Naive Bayes Classifier

In [ ]:
# Naive Bayes-Modell erstellen
nb_classifier = GaussianNB()

# Modell auf den resamplten Trainingsdatensatz anpassen
nb_classifier.fit(X_train_resampled, y_train_resampled)

# Vorhersagen auf dem Validierungsset machen
y_prob_val_nb = nb_classifier.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_nb)

Here we see that the naive Bayes classifier is not much better than a model that would randomly choose between the two classes. Since we have set the recall to 0.9 and the model cannot distinguish well between the two classes, class 1 is predicted significantly more often, which means that the number of false negatives is low and therefore the recall is high.

### 3.3 Decision Tree

#### 3.3.1 Untuned Decision Trees

+ First we will use an untuned decision tree as a model and then search for the optimal hyperparameters with RandomizedSearchCV and cross-validation.
+ First of all, for example, we will plot the decision tree (but not with max depth)

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)

fig, ax = plt.subplots(figsize=(12, 8))
plot_tree(tree, label="root", filled=True, max_depth=1, 
          feature_names=X.columns, ax=ax);

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train_resampled, y_train_resampled)

# Wahrscheinlichkeiten für die positive Klasse (Klasse 1) auf dem Validierungsset erhalten
y_prob_val_dt1 = tree.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_dt1)

#### 3.3.2 Hyperprameter Tuning - Decision Trees

In [ ]:
# Parametergitter für die Anzahl der Bäume
param_grid = {
    'criterion': ["gini","entropy"],
    'max_depth': [10,20,30,None],
    'min_samples_split': [2,5,7,9],
    'min_samples_leaf': [2,5,7,9]
}

tree = DecisionTreeClassifier()
grid_search = RandomizedSearchCV(tree, param_grid, cv=3, n_iter=10)
grid_search.fit(X_train_resampled, y_train_resampled)

# Beste Parameter abrufen
best_estimator = grid_search.best_estimator_
y_prob_val_dt2 = best_estimator.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_dt2)


### 3.4 Logistic Regression

#### 3.4.1 Basic Logistic Regression

In [ ]:
# Create and fit the logistic regression model
log = LogisticRegression(max_iter=1000, random_state=42)
log.fit(X_train, y_train)

# Plot ROC curve
y_prob_val_LR1 = log.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_LR1)

#### 3.4.2 Hyperparameter Tuning - Logistic Regression

In [ ]:
# Define logistic regression models with L1 and L2 regularization
lasso_model = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, tol=1e-3, n_jobs=-1)
ridge_model = LogisticRegression(penalty='l2', solver='saga', max_iter=1000, tol=1e-3, n_jobs=-1)

# Define the parameter grid to search
param_grid = {'C': [0.01, 0.1, 1, 10]}

# Define the scoring metric (ROC-AUC) and specify that you want to maximize it
scoring = {'AUC': 'roc_auc', 'Recall': make_scorer(recall_score, pos_label=1)}

# Perform randomized search with cross-validation
lasso_random_search = RandomizedSearchCV(lasso_model, param_distributions=param_grid, n_iter=3, scoring=scoring, cv=3, refit='Recall', n_jobs=-1)
ridge_random_search = RandomizedSearchCV(ridge_model, param_distributions=param_grid, n_iter=3, scoring=scoring, cv=3, refit='Recall', n_jobs=-1)

# Fit the models with the best parameters
lasso_random_search.fit(X_train_resampled, y_train_resampled)
ridge_random_search.fit(X_train_resampled, y_train_resampled)

# Get the best models
best_lasso_model = lasso_random_search.best_estimator_
best_ridge_model = ridge_random_search.best_estimator_

y_prob_val_lasso = best_lasso_model.predict_proba(X_val)[:, 1]
y_prob_val_ridge = best_ridge_model.predict_proba(X_val)[:, 1]

plot_results(y_prob_val_lasso)
plot_results(y_prob_val_ridge)

### 3.5 Neural Network

We tried two methods to oversample our inbalanced dataset. But we figured out that the neuronal network performs slighly worse than without oversampling. Therefore we won't use the oversampling method for neuronal networks.

### 3.5.1 Model 1 (Model for testing out different layers and neuron counts)

In [ ]:
X_train.shape[1]

In [ ]:
# baselinemodel2 with a hidden layer consisting of 500 neurons
def create_baseline2():
	# create model
	model = Sequential()
	model.add(Dense(500, input_shape=(1523,), activation='relu')) 
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

In [ ]:
# evaluate baseline model with standardized dataset
estimator2 = KerasClassifier(model=create_baseline2, epochs=8, batch_size=20, verbose=0, metrics=[recall_score])
kfold = StratifiedKFold(n_splits=3, shuffle=True)
results = cross_val_score(estimator2, X_train, y_train, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
y_prob_val_nn2 = estimator2.predict_proba(X_val)[:,1]	

In [ ]:
plot_results(y_prob_val_nn2)

We can observe that this type of network has espacially a big std. Let's compare this network with another one which has less neurons in the hidden layer and compare both networks. We can observe that the network performs slightly better on onseen data but it seems that it performs better much better in training. The reason for this could be that we slightly overfit the network on the training data. We can conclude that the performance of the second network is as good as the first but computational cost is much lower. Let's see how a neronal network with two hidden layers performs. As we could conclude from this experiment, we can observe that with less neurons in the hidden layer, the network performs better on the training data and slightly better on the test data. Therfore we will try to set the neurons in the second hidden layer to 250 and see how the network performs. The idea here is that the network is given the opportunity to model all input variables before being bottlenecked and forced to halve the representational capacity.

### 3.5.2 Hyperparameter Tuning 1

In [ ]:
def create_baseline7(hp):
    model = Sequential()
    for i in range(hp.Int('layers', 2, 6)):
        model.add(Dense(
            units=hp.Int('units_' + str(i), 20, 300, step=35), input_shape=(1523,),
            activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])
        ))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner1 = RandomSearch(
    create_baseline7,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=6,
    directory='my_dir',
    project_name='tuner1'
)

tuner1.search(X_train, y_train, epochs=5)

In [ ]:
tuner1.results_summary()

In [ ]:
best_models = tuner1.get_best_models(num_models=2) # we will select the best 3 models and want to see how they perform on unseen data
y_prob_val_nn3 = best_models[0].predict_proba(X_val)[:,1]
y_prob_val_nn4 = best_models[1].predict_proba(X_val)[:,1]

In [ ]:
# let's call our plot function to plot the ROC of each model compared to the recall
plot_results(y_prob_val_nn3)
plot_results(y_prob_val_nn4)

### 3.5.3 Hyperparameter Tuning 2

In [ ]:
def create_baseline8(hp):
    model = Sequential()
    for i in range(hp.Int('layers', 2, 10)):
        model.add(Dense(
            units=hp.Int('units_' + str(i), 10, 200, step=20), input_shape=(1523,),
            activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])
        ))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(curve='ROC')])
    return model

tuner8 = RandomSearch(
    create_baseline8,
    objective=Objective('auc', direction='max'),
    max_trials=5,
    executions_per_trial=5,
    directory='my_dir',
    project_name='tuner2'
)

In [ ]:
tuner8.search(X_train, y_train, epochs=5)

In [ ]:
tuner8.results_summary()

In [ ]:
best_model8 = tuner8.get_best_models(num_models=2)  # select the three three best models with corresponding hyperparameters
y_prob_val_nn5 = best_model8[0].predict_proba(X_val)[:,1]
y_prob_val_nn6 = best_model8[1].predict_proba(X_val)[:,1]

In [ ]:
# let's call our plot function to plot the ROC of each model compared to the recall
plot_results(y_prob_val_nn5)
plot_results(y_prob_val_nn6)

### 3.5.4 Hyperparameter Tuning 3

In [ ]:
# Definiere Hyperparameter für die Lernrate und Batch-Größe
def create_baseline9(hp):
    model = Sequential()
    for i in range(hp.Int('layers', 2, 15)):
        model.add(Dense(
            units=hp.Int('units_' + str(i), 10, 600, step=25), input_shape=(1523,),
            activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])
        ))
    model.add(Dense(1, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_batch_size = hp.Choice('batch_size', values=[32, 64, 128])

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=hp_learning_rate), metrics=[tf.keras.metrics.AUC(curve='ROC')])
    return model


tuner9 = RandomSearch(
    create_baseline9,
    objective=Objective('auc', direction='max'),
    max_trials=5,
    executions_per_trial=5,
    directory='my_dir',
    project_name='tuner3'
)

In [ ]:
tuner9.search(X_train, y_train, epochs=5,)

In [ ]:
tuner9.results_summary()

In [ ]:
# we are now interested in the best 3 models with the corresponding hyperparameters.
best_model9 = tuner9.get_best_models(num_models=2)
y_prob_val_nn7 = best_model9[0].predict_proba(X_val)[:,1]
y_prob_val_nn8 = best_model9[1].predict_proba(X_val)[:,1]

In [ ]:
# let's call our plot function to plot the ROC of each model compared to the recall
plot_results(y_prob_val_nn7)
plot_results(y_prob_val_nn8)

## 4. Conclusion

To conclude we compar the roc curves of all models to choose the best one for out final prediction.

In [ ]:
# Funktion zur Erstellung und Anzeige von ROC-Kurven
def plot_roc_curve(y_true, y_probs, label):
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    # Plot erstellen
    plt.plot(fpr, tpr, lw=2, label=f'{label} (AUC = {roc_auc:.2f})')

y_prob_list = [y_prob_val_rf1, y_prob_val_rf2, y_prob_val_rf3,
               y_prob_val_nb, y_prob_val_dt1, y_prob_val_dt2,
               y_prob_val_LR1, y_prob_val_lasso, y_prob_val_ridge,
               y_prob_val_nn2, y_prob_val_nn3, y_prob_val_nn4, y_prob_val_nn5, y_prob_val_nn6, y_prob_val_nn7, y_prob_val_nn8]

# Plot erstellen
plt.figure(figsize=(8, 8))

for i, y_prob in enumerate(y_prob_list):
    plot_roc_curve(y_val, y_prob, i)

# Zufallsraten
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Zufallsraten')

# Achsenbeschriftungen und Titel hinzufügen
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison on Validation Set')
plt.legend(loc='lower right')

# Anzeigen der Grafik
plt.show()

As the graph shows, many of the models perform very similar. We chose the random forest classifier with tuned hyperparameters for our final prediction.

In [ ]:
best_model = randomforestcv

y_pred = best_model.predict(X_test)

#z_test = np.array(z_test.str[2:].astype(int))

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Extract the indices of false negatives
fn_indices = np.where((y_test == True) & (y_pred == False))[0]

# Use the indices to get the corresponding values from z_test
fn_values = z_test[fn_indices]

# Plot the distribution
plt.figure(figsize=(8, 6))
plt.hist(fn_values, bins=[1, 2, 3, 4, 5], align='left', rwidth=0.8, color='skyblue', edgecolor='black')
plt.title('Distribution of True Values for False Negatives')
plt.xlabel('Categories')
plt.ylabel('Count')
plt.xticks([1, 2, 3, 4], ['Death', 'Heavy Injury', 'Light Injury', 'Property Damage'])
plt.show()

plot_results(best_model.predict_proba(X_test)[:,1], y_true=y_test)


To conclude, the graph above shows, that we predict about 200 false negatives, but most of them luckily only resultet in light personal injuries.

## 5. Sources

- Main-Dataset: https://data.stadt-zuerich.ch/dataset/sid_dav_strassenverkehrsunfallorte  startzeit, wahrscheinlich auch zeitumstellung
- Weather data: https://data.stadt-zuerich.ch/dataset/ugz_meteodaten_stundenmittelwerte  startzeit, 10Uhr Sommer = 10Uhr i.R., 10Uhr Winter = 11Uhr i.R.
- Traffic volume data: https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031  startzeit, vorstellen: es fehlt 2Uhr, rückstellen: es gibt 2 Werte für 2Uhr
- Streets network and speedlimit data: https://data.stadt-zuerich.ch/dataset/geo_signalisierte_geschwindigkeiten  
- Pedestrian and bicycle volume data: https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo  startzeit, vorstellen: es fehlt 2Uhr, rückstellen: es gibt 2 Werte für 2Uhr
- Trafic-areas: https://data.stadt-zuerich.ch/dataset/geo_verkehrszonen 
- Public transport stops: https://data.stadt-zuerich.ch/dataset/ktzh_haltestellen_des_oeffentlichen_verkehrs___ogd_
- Public garages: https://www.stadt-zuerich.ch/geodaten/
- Pedestrian and Bicycle network: https://data.stadt-zuerich.ch/dataset/geo_fuss__und_velowegnetz
- Timeshift information: https://www.zeitumstellung.in
- Holidays Information: https://www.stadt-zuerich.ch/portal/de/index/jobs/anstellungsbedingungen/ferien-urlaub-betriebsferientage/feiertage-betriebsferientage.html and https://www.feiertagskalender.ch/kalender.php?geo=3055&jahr=2026&hl=de&klasse=4&ft_id=20
